In [1]:
from iGTP_Linear import *
from iGTP_model_modified_scvi_sample_final_update import *
from preprocess import *
from learning_utilies import *
import os 
import scanpy as sc
import pickle5 as pickle

In [2]:
ppi_file_list=os.listdir('/home/iGTP/ppi/')
ppi_file_list=[i for i in ppi_file_list if'.ipynb_checkpoints' not in i]
ppi_file_list.sort()
print(ppi_file_list)
tp_file_list=os.listdir('/home/iGTP/TP/')
tp_file_list=[i for i in tp_file_list if'.ipynb_checkpoints' not in i]
tp_file_list=[i for i in tp_file_list if 'brain' in i or 'GO' in i]
tp_file_list.sort()
print(tp_file_list)
data_file_list=os.listdir('/home/iGTP/iGTP_data')
data_file_list=[i for i in data_file_list if'._' not in i]
data_file_list=[i for i in data_file_list if'.h5ad' in i]
data_file_list.sort()
print(data_file_list)

['PC.txt', 'PC_homo_v11.txt', 'PPI_compiled.txt']
['GO_merged.gmt', 'GO_merged_modify', 'brain_cell_type_function_AD_pathways.gmt']
['Adamson2016.h5ad', 'GBM.h5ad', 'Microglia_ipsc_only_2022.h5ad', 'NormanWeissman2019_filtered.h5ad', 'ShifrutMarson2018.h5ad', 'TianKampmann2019_day7neuron.h5ad', 'covid19.h5ad', 'covid_pbmc_ifn_test.h5ad', 'covid_pbmc_ifn_train.h5ad', 'covid_pbmc_ifn_train_test.h5ad', 'pbmc3k.h5ad', 'pbmc_IFN.h5ad', 'pbmc_IFN_1.h5ad', 'pbmc_IFN_filtered.h5ad', 'pbmc_IFN_filtered_1.h5ad', 'resilience.h5ad', 'resilience_ips.h5ad']


In [3]:
data_file_list[9]

'covid_pbmc_ifn_train_test.h5ad'

In [4]:
args={}
args['tp_overlap_fraction']=0.1
args['encoder_layer_list']=[1500]
args['beta']=0.00005
args['encoder_normal']='batch'
args['init_type']='pos_uniform'
args['recon_loss']='mse'
args['drop_out']=0.2
args['eps']=0.001
args['cv_fold']=5
args['learning_rate']=0.001
args['learning_rate_weight_decay']=0.000054
args['n_epochs']=500
args['train_patience']=10
args['test_patience']=10
args['data_num_workers']=1
args['save_fold']=True
args['save_best']=True
args['anneal_start']=None
args['anneal_time']=None
args['sc_data_path']='/home/iGTP/iGTP_data/'+data_file_list[9]
args['tp_file_path']='/home/iGTP/TP/'+tp_file_list[0]
args['ppi_file_path']='/home/iGTP/ppi/'+ppi_file_list[1]
args['model_dir']='./save_model/'+data_file_list[9].replace('.h5ad','')+'/'
args['vb_nu']=15000
args['model_prefix']=tp_file_list[0].replace('.gmt','')+'_'+ppi_file_list[1].replace('.txt','')+args['recon_loss']+'_'+args['init_type']+'_'+str(args['vb_nu'])+'batch_inter'
args['batch_size']=8000
args['drop_last_batch']=False
args['device_nu']=1
args['using_gpu']=True
args['using_parallel']=True
args['z_sample']=1

In [5]:
adata = sc.read(args['sc_data_path'])
#adata=adata[adata.obs.dataset!='Test']
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
adata.var['mt'] = adata.var_names.str.startswith('MT-') 
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
adata = adata[adata.obs.n_genes_by_counts < 10000, :]
adata = adata[adata.obs.pct_counts_mt < 20, :]
adata = adata[adata.obs.total_counts < 50000, :]
sc.pp.normalize_total(adata, target_sum=1e5)
sc.pp.log1p(adata)
#sc.pp.highly_variable_genes(adata, n_top_genes=args['vb_nu'])
print(adata.X.shape)
#adata.raw = adata
#adata = adata[:, adata.var.highly_variable]
print(adata.X.shape)

/usr/local/lib/python3.10/dist-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


(761304, 21555)
(761304, 21555)


In [6]:
adata.raw = adata
#adata = adata[:, adata.var.highly_variable]

In [7]:
adata_train=adata[adata.obs.dataset!='Test'].copy()
adata_test=adata[adata.obs.dataset=='Test'].copy()
print(adata.X.shape)

(761304, 21555)


In [ ]:
data_x_s_train,mask_list_dict,args,seq_gene_df,seq_gene_df_su, TP_df, ppi_df_1,data_train=pre_process(args,adata_train)
data_x_s_test,_,_,_,_, _, _,data_test=pre_process(args,adata_test)
all_data_df={}
all_data_df['data_x_s_train']=data_x_s_train
all_data_df['data_x_s_test']=data_x_s_test
all_data_df['mask_dict']=mask_list_dict
all_data_df['args']=args
all_data_df['seq_gene_df']=seq_gene_df
all_data_df['seq_gene_df_su']=seq_gene_df_su
all_data_df['TP_df']=TP_df
all_data_df['ppi_df_1']=ppi_df_1
all_data_df['data_train']=data_train
all_data_df['data_test']=data_test
args['model_data_dir']=args['model_dir']+'processed_data/'
if not os.path.exists(args['model_data_dir']):
            # Create the directory if it doesn't exist
    os.makedirs(args['model_data_dir'],exist_ok=True)
    print("Directory created successfully.")
else:
    print("Directory already exists.")
model_data_file=args['model_data_dir']+args['model_prefix']+'.pkl'
with open(model_data_file, 'wb') as file:
    pickle.dump(all_data_df, file)
kfold=KFoldTorch(args) 
kfold.train_kfold(iGTP,mask_list_dict,data_x_s_train)

13634
Number of Gene 13000 in PPI
Create TP->Gene mask
Number of TP 1303, and Gene 13634
Create Gene->Gene mask
Number of Gene 13634, and Gene 13634
The number of gene in TP Yes was 13634.00 
13634
Number of Gene 13000 in PPI
Create TP->Gene mask
Number of TP 1303, and Gene 13634
Create Gene->Gene mask
Number of Gene 13634, and Gene 13634
The number of gene in TP Yes was 13634.00 
Directory already exists.
Model(s) will be saved at ./save_model/covid_pbmc_ifn_train_test/ using GO_merged_PC_homo_v11mse_pos_uniform_15000batch_inter as prefix


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Training fold 0
[Epoch 1] | loss: 1958682.020 | test_loss: 11085.631 |
[Epoch 2] | loss: 11401.543 | test_loss: 10755.316 |
[Epoch 3] | loss: 11302.775 | test_loss: 10734.275 |
[Epoch 4] | loss: 11249.574 | test_loss: 10647.429 |
[Epoch 5] | loss: 11197.260 | test_loss: 10630.798 |
[Epoch 6] | loss: 11160.261 | test_loss: 10626.980 |
[Epoch 7] | loss: 11140.116 | test_loss: 10582.740 |
[Epoch 8] | loss: 11118.661 | test_loss: 10556.918 |
[Epoch 9] | loss: 11099.512 | test_loss: 10543.530 |
[Epoch 10] | loss: 11081.680 | test_loss: 10504.743 |
[Epoch 11] | loss: 11064.121 | test_loss: 10508.062 |
[Epoch 12] | loss: 11054.873 | test_loss: 10464.135 |
[Epoch 13] | loss: 11041.996 | test_loss: 10500.806 |
[Epoch 14] | loss: 11033.132 | test_loss: 10489.459 |
[Epoch 15] | loss: 11024.122 | test_loss: 10425.651 |
[Epoch 16] | loss: 11017.415 | test_loss: 10434.520 |
[Epoch 17] | loss: 11006.791 | test_loss: 10416.946 |
[Epoch 18] | loss: 10998.824 | test_loss: 10411.961 |
[Epoch 19] | loss: 

[Epoch 154] | loss: 9987.822 | test_loss: 9252.729 |
[Epoch 155] | loss: 9985.267 | test_loss: 9250.324 |
[Epoch 156] | loss: 9982.688 | test_loss: 9249.112 |
[Epoch 157] | loss: 9980.026 | test_loss: 9241.270 |
[Epoch 158] | loss: 9977.525 | test_loss: 9238.505 |
[Epoch 159] | loss: 9974.801 | test_loss: 9236.348 |
[Epoch 160] | loss: 9971.910 | test_loss: 9232.939 |
[Epoch 161] | loss: 9969.543 | test_loss: 9230.462 |


In [ ]:
kfold=KFoldTorch(args) 
kfold.train_kfold(iGTP,mask_list_dict,data_x_s_train)

In [ ]:
 if torch.cuda.is_available() and self.args['using_gpu'] is True:
            
            # Get the number of available GPUs
            num_gpus = torch.cuda.device_count()

            # Choose the GPU you want to use (e.g., GPU 0)
            gpu_number = self.args['device_nu']

            # Set the device to the selected GPU
            if self.args['using_parallel'] is True:
                self.args['device']=[i for i in range(gpu_number)]
            else:
                self.args['device'] = torch.device(f"cuda:{gpu_number}")

        # Move the model to the selected GPU
        else:
            self.args['device'] = torch.device("cpu")
        
        self.model = MyModel(self.args,mask_list_dict)

In [ ]:
args['device']=[i for i in range(3)]

In [ ]:
model = iGTP(args,mask_list_dict)

In [ ]:
import torch.nn as nn

In [ ]:
#model = nn.DataParallel(model, device_ids=args['device'])
            #self.model = self.model.to(f'cuda:{self.args["device"][0]}')
            
dummy_input = torch.randn(2, 9153)#.to(args['device'][0])
model(dummy_input)  
model = nn.DataParallel(model, device_ids=args['device'])
model.to(args['device'][0])
optimizer = optim.Adam(model.parameters(), lr=args['learning_rate'], weight_decay=args['learning_rate_weight_decay'])
    

In [ ]:
train_loader = torch.utils.data.DataLoader(data_x_s_train, batch_size=9000, shuffle=True,\
                                                       drop_last=args['drop_last_batch'], num_workers=args['data_num_workers'])
            

In [ ]:
x_train=next(iter(train_loader))

In [ ]:
x_train.shape

In [ ]:
model

In [ ]:
x_train.float().shape

In [ ]:
model(x_train.float())

In [ ]:
{c:{} for c in range(args['cv_fold'])}

In [ ]:
args

In [ ]:
kfold.args

In [ ]:
[i for i in range(3)]